<a href="https://colab.research.google.com/github/SAR-bang/RecommenderSystem/blob/main/Recommendaersystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import files
data_to_load = files.upload()

In [10]:
import pandas as pd
import numpy as np
from scipy import sparse
df = pd.read_csv("post_data.csv")
df.head()
df = df.rename(columns={"Valuable ":"rating"})


In [10]:
feed = df.groupby('rating')
feed.groups
feed.count()

In [11]:
feed.nunique()

,user_id,post_id,time_stamp
rating,,,
1.0,464,3086,4314
2.0,455,3093,4268
3.0,470,3161,4381
4.0,463,3105,4331
5.0,465,3062,4267


In [12]:
df.isnull().sum() # finds null value in each column

user_id       0
post_id       1
time_stamp    1
rating        1
dtype: int64

In [13]:
df.dropna(inplace=True)

In [14]:
df.isnull().sum()

user_id       0
post_id       0
time_stamp    0
rating        0
dtype: int64

In [15]:
# mapping the values to 0 and 1 assuming values between1,2 and 3,4,5

df['rating']= df['rating'].map({1:0,2:0,3:1,4:1,5:1})

In [16]:
df['rating'].head()

0    0
1    0
2    0
3    0
4    1
Name: rating, dtype: int64

In [17]:
# creating a new  df for UIR (user item ratings)
# s1 = df['user_id']
# s2 = df['post_id']
# s3 = df['rating']
# left = pd.DataFrame([s3],index=[s1])

# right = pd.DataFrame([s3],index=[s2])

In [32]:
percent = int(len(df)*0.8)
train_data = df[:percent]
test_data = df[percent:]

In [33]:
no_rated_post_per_user = train_data.groupby(by="user_id")["rating"].count().sort_values(ascending=False)
no_rated_post_per_user.head()

user_id
5eece14ffc13ae6609000157    89
5eece14ffc13ae6609000117    87
5eece14ffc13ae66090001bb    81
5eece14ffc13ae66090001a9    80
5eece14ffc13ae6609000133    79
Name: rating, dtype: int64

In [34]:
no_ratings_per_post = train_data.groupby(by = "post_id")["rating"].count().sort_values(ascending = False)
no_ratings_per_post.head()

post_id
581460074.0    11
140309295.0    10
737136135.0    10
262223498.0    10
433010257.0     9
Name: rating, dtype: int64

In [35]:
def get_user_item_sparse_matrix(df):
    sparse_data = sparse.csr_matrix((df.rating, (df.user_id, df.post_id)))
    return sparse_data

In [36]:
train_sparse_data = get_user_item_sparse_matrix(train_data)

TypeError: ignored